In [1]:
import csv
import cv2
import numpy as np
import tensorflow as tf
import os

In [2]:
WINDOW = "Hand Tracking"
PALM_MODEL_PATH = "./palm_detection_without_custom_op.tflite"
LANDMARK_MODEL_PATH = "./hand_landmark.tflite"
#LANDMARK_MODEL_PATH = "./hand_landmark_3d.tflite"
ANCHORS_PATH = "./anchors.csv"

In [3]:
class HandTracker():
    
    def __init__(self, palm_model,
                joint_model,
                anchors_path,
                box_enlarge=1.5,
                box_shift=0.2):
        
        self.box_shift = box_shift
        self.box_enlarge = box_enlarge

        self.interp_palm = tf.lite.Interpreter(palm_model)
        self.interp_palm.allocate_tensors()
        self.interp_joint = tf.lite.Interpreter(joint_model)
        self.interp_joint.allocate_tensors()

        # reading the SSD anchors
        with open(anchors_path, "r") as csv_f:
            self.anchors = np.r_[
                [x for x in csv.reader(csv_f, quoting=csv.QUOTE_NONNUMERIC)]
            ]
        # reading tflite model paramteres
        output_details = self.interp_palm.get_output_details()
        input_details = self.interp_palm.get_input_details()

        self.in_idx = input_details[0]['index']
        self.out_reg_idx = output_details[0]['index']
        self.out_clf_idx = output_details[1]['index']

        self.in_idx_joint = self.interp_joint.get_input_details()[0]['index']
        self.out_idx_joint = self.interp_joint.get_output_details()[0]['index']

        # 90° rotation matrix used to create the alignment trianlge
        self.R90 = np.r_[[[0,1],[-1,0]]]

        # trianlge target coordinates used to move the detected hand
        # into the right position
        self._target_triangle = np.float32([
                        [128, 128],
                        [128,   0],
                        [  0, 128]
                    ])
        self._target_box = np.float32([
                        [  0,   0, 1],
                        [256,   0, 1],
                        [256, 256, 1],
                        [  0, 256, 1],
                    ])

    def _get_triangle(self, kp0, kp2, dist=1):
        dir_v = kp2 - kp0
        dir_v /= np.linalg.norm(dir_v)

        dir_v_r = dir_v @ self.R90.T
        return np.float32([kp2, kp2+dir_v*dist, kp2 + dir_v_r*dist])

    @staticmethod
    def _triangle_to_bbox(source):
        # plain old vector arithmetics
        bbox = np.c_[
            [source[2] - source[0] + source[1]],
            [source[1] + source[0] - source[2]],
            [3 * source[0] - source[1] - source[2]],
            [source[2] - source[1] + source[0]],
        ].reshape(-1,2)
        return bbox

    @staticmethod
    def _im_normalize(img):
         return np.ascontiguousarray(2 * ((img / 255) - 0.5).astype('float32'))

    @staticmethod
    def _sigm(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def _pad1(x):
        return np.pad(x, ((0, 0), (0, 1)), constant_values=1, mode='constant')

    def predict_joints(self, img_norm):
        self.interp_joint.set_tensor(self.in_idx_joint, img_norm.reshape(1,256,256,3))
        self.interp_joint.invoke()

        joints = self.interp_joint.get_tensor(self.out_idx_joint)
        
        return joints.reshape(-1,2)
        #return joints.reshape(-1, 3)

    # predict hand location and landmarks
    def detect_hand(self, img_norm):
        self.interp_palm.set_tensor(self.in_idx, img_norm[None])
        self.interp_palm.invoke()

        out_reg = self.interp_palm.get_tensor(self.out_reg_idx)[0]
        out_clf = self.interp_palm.get_tensor(self.out_clf_idx)[0,:,0]
        
        detecion_mask = self._sigm(out_clf) > 0.85
        candidate_detect = out_reg[detecion_mask]
        candidate_anchors = self.anchors[detecion_mask]

        if candidate_detect.shape[0] == 0:
            #print("No hands found")
            return None, None
        
        max_idx = np.argmax(candidate_detect[:, 3])
        
        dx,dy,w,h = candidate_detect[max_idx, :4]
        center_wo_offst = candidate_anchors[max_idx,:2] * 256
        
        keypoints = center_wo_offst + candidate_detect[max_idx,4:].reshape(-1,2)
        side = max(w,h) * self.box_enlarge
        
        source = self._get_triangle(keypoints[0], keypoints[2], side)
        source -= (keypoints[0] - keypoints[2]) * self.box_shift
        return source, keypoints

    #normalize image to 256x256
    def preprocess_img(self, img):
        shape = np.r_[img.shape]
        pad = (shape.max() - shape[:2]).astype('uint32') // 2
        img_pad = np.pad(img, ((pad[0],pad[0]), (pad[1],pad[1]), (0,0)), mode='constant')
        
        img_small = cv2.resize(img_pad, (256, 256))
        img_small = np.ascontiguousarray(img_small)

        img_norm = self._im_normalize(img_small)
        return img_pad, img_norm, pad

    def __call__(self, img):
        img_pad, img_norm, pad = self.preprocess_img(img)

        source, keypoints = self.detect_hand(img_norm)
        if source is None:
            return None, None

        # calculating transformation from img_pad coords
        # to img_landmark coords (cropped hand image)
        scale = max(img.shape) / 256
        Mtr = cv2.getAffineTransform(
            source * scale,
            self._target_triangle
        )

        img_landmark = cv2.warpAffine(
            self._im_normalize(img_pad), Mtr, (256,256)
        )

        joints = self.predict_joints(img_landmark)

        # adding the [0,0,1] row to make the matrix square
        Mtr = self._pad1(Mtr.T).T
        Mtr[2,:2] = 0

        Minv = np.linalg.inv(Mtr)

        # projecting keypoints back into original image coordinate space
        kp_orig = (self._pad1(joints[:, :2]) @ Minv.T)[:,:2]
        box_orig = (self._target_box @ Minv.T)[:,:2]
        kp_orig -= pad[::-1]
        box_orig -= pad[::-1]

        return kp_orig, box_orig

In [18]:
POINT_COLOR = (0, 255, 0)
CONNECTION_COLOR = (255, 0, 0)
THICKNESS = 2

cv2.namedWindow(WINDOW)
capture = cv2.VideoCapture(0)

if capture.isOpened():
    hasFrame, frame = capture.read()
else:
    hasFrame = False

connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

detector = HandTracker(
    PALM_MODEL_PATH,
    LANDMARK_MODEL_PATH,
    ANCHORS_PATH,
    box_shift=0.2,
    box_enlarge=1.3
)

while hasFrame:
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    points, bbox = detector(image)
    
    if points is not None:
        '''
        for point in points:
            x, y = point
            cv2.circle(frame, (int(x), int(y)), THICKNESS*2, POINT_COLOR, THICKNESS)
        '''
        
        cv2.line(frame, (int(bbox[0][0]), int(bbox[0][1])),
                 (int(bbox[1][0]), int(bbox[1][1])), POINT_COLOR, 1)
        cv2.line(frame, (int(bbox[1][0]), int(bbox[1][1])),
                 (int(bbox[2][0]), int(bbox[2][1])), POINT_COLOR, 1) 
        cv2.line(frame, (int(bbox[2][0]), int(bbox[2][1])),
                 (int(bbox[3][0]), int(bbox[3][1])), POINT_COLOR, 1) 
        cv2.line(frame, (int(bbox[3][0]), int(bbox[3][1])),
                 (int(bbox[0][0]), int(bbox[0][1])), POINT_COLOR, 1)
        
        '''
        for connection in connections:
            x0, y0 = points[connection[0]]
            x1, y1 = points[connection[1]]
            cv2.line(frame, (int(x0), int(y0)), (int(x1), int(y1)), CONNECTION_COLOR, THICKNESS)
        '''
        
    cv2.imshow(WINDOW, frame)
    hasFrame, frame = capture.read()
    key = cv2.waitKey(20)
    if key == 27:
        break

capture.release()
cv2.destroyAllWindows()

In [ ]:
#        8   12  16  20
#        |   |   |   |
#        7   11  15  19
#    4   |   |   |   |
#    |   6   10  14  18
#    3   |   |   |   |
#    |   5---9---13--17
#    2    \         /
#     \    \       /
#      1    \     /
#       \    \   /
#        ------0-

In [4]:
PATH = "./data/"

In [6]:
output_file = open("output.txt", "w")

detector = HandTracker(
    PALM_MODEL_PATH,
    LANDMARK_MODEL_PATH,
    ANCHORS_PATH,
    box_shift=0.2,
    box_enlarge=1.3
)

for name_dir  in os.listdir(PATH):
    print(name_dir)
    output_file.write(str(name_dir) + "\t" + str(len(os.listdir(PATH + name_dir))) + "\t")
    for name_file in os.listdir(PATH + name_dir):
        image = cv2.imread(PATH + name_dir + "/" + name_file, flags=cv2.IMREAD_COLOR)
        points, bbox = detector(image)
        
        if points is not None:
            for p in points:
                output_file.write(str(p[0]) + " " + str(p[1]) + " ")
        
    output_file.write("\n")

8571
12841
1
4677


In [1]:
import pandas as pd

In [2]:
STOP_SIGN = "Stop Sign"
SWIPING_LEFT = "Swiping Left"
SWIPING_RIGHT = "Swiping Right"
SWIPING_DOWN = "Swiping Down"
SWIPING_UP = "Swiping Up"
THUMB_UP = "Thumb Up"
NO_GESTURE = "No gesture"
DOING_OTHER_THINGS = "Doing other things"

In [3]:
train_df = pd.read_csv("labeles/jester-v1-train.csv", sep=";", header=None)
#test_df = pd.DataFrame("labeles/jester-v1-test.csv", sep="\t", header=None)
val_df = pd.read_csv("labeles/jester-v1-validation.csv", sep=";", header=None)

In [4]:
out_df = train_df[train_df[1] == STOP_SIGN]
out_df = pd.concat([out_df, val_df[val_df[1] == STOP_SIGN]])

out_df = pd.concat([out_df, train_df[train_df[1] == SWIPING_LEFT]])
out_df = pd.concat([out_df, val_df[val_df[1] == SWIPING_LEFT]])

out_df = pd.concat([out_df, train_df[train_df[1] == SWIPING_RIGHT]])
out_df = pd.concat([out_df, val_df[val_df[1] == SWIPING_RIGHT]])

out_df = pd.concat([out_df, train_df[train_df[1] == SWIPING_DOWN]])
out_df = pd.concat([out_df, val_df[val_df[1] == SWIPING_DOWN]])

out_df = pd.concat([out_df, train_df[train_df[1] == SWIPING_UP]])
out_df = pd.concat([out_df, val_df[val_df[1] == SWIPING_UP]])

out_df = pd.concat([out_df, train_df[train_df[1] == THUMB_UP]])
out_df = pd.concat([out_df, val_df[val_df[1] == THUMB_UP]])

out_df = pd.concat([out_df, train_df[train_df[1] == NO_GESTURE]])
out_df = pd.concat([out_df, val_df[val_df[1] == NO_GESTURE]])

out_df = out_df.sample(frac=1).reset_index(drop=True)
out_df.to_csv("labeles.csv", index=False, sep="\t", header=None)

In [5]:
out_df.shape

(33329, 2)